In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Dropdown, interactive
from IPython.display import display, clear_output
from commpy.channels import awgn
from scipy.signal import convolve
import time

# Function to calculate bit errors in coherent FSK
def fsk_errors(bps, Nsymb, ns, EbNo):
    # Input parameters
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    fc = 2 * M * BR  # RF frequency

    # Derived parameters
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period

    # M frequencies in "coherent" distance (BR)
    f = fc + (BR/2) * (np.arange(1, M + 1) - (M + 1) / 2)

    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # awgn channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)
    x = y.reshape((Nsymb, bps))

    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)  # oversampling time vector

    # FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        fk = f[int(''.join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s.append(np.sin(2 * np.pi * fk * tk))
    s = np.concatenate(s)

    # add noise to the FSK (passband) signal
    s = awgn(s, SNR)

    # FSK receiver
    xr = []
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        smi = []
        for fi in f:
            si = np.sin(2 * np.pi * fi * tk)
            smi.append(np.sum(sk * si))
        j = np.argmax(smi)
        xr.append([int(bit) for bit in bin(j)[2:].zfill(bps)])
    xr = np.array(xr).reshape((Nsymb, bps))

    # count errors
    errors = np.sum(x != xr)
    return errors

# Non-coherent FSK error calculation
def fsk_errors_non_coh(bps, Nsymb, ns, EbNo):
    M = 2 ** bps  # number of different symbols
    BR = 1  # Baud Rate
    fc = 2 * M * BR  # RF frequency

    # Derived parameters
    nb = bps * Nsymb  # number of simulated data bits
    T = 1 / BR  # one symbol period
    Ts = T / ns  # oversampling period
    f = fc + BR * (np.arange(1, M + 1) - (M + 1) / 2)  # M frequencies
    # Calculate the maximum frequency
    fmax = np.max(f)

    # Recalculate ns to ensure Fs = ns * BR > 2 * fmax
    Fs = 2 * fmax
    ns = int(np.ceil(Fs / BR)) + 10

    # Recalculate the oversampling period
    Ts = T / ns

    # AWGN channel
    SNR = EbNo + 10 * np.log10(bps) - 10 * np.log10(ns / 2)  # in dB

    # input data bits
    y = np.random.randint(0, 2, nb)  # Ensure integer dimensions
    x = y.reshape((-1, bps))
    t = np.arange(0, len(x) * T, T)  # time vector on the T grid
    tks = np.arange(0, T, Ts)

    # FSK signal
    s = []
    A = np.sqrt(2 / T / ns)
    for k in range(len(x)):
        fk = f[int("".join(map(str, x[k])), 2)]
        tk = (k * T) + tks
        s = np.concatenate((s, np.sin(2 * np.pi * fk * tk)))

    # add noise to the FSK (passband) signal
    s = awgn(s, SNR)

    # FSK receiver
    # Non-coherent demodulation
    xr = []
    for k in range(len(s) // ns):
        tk = (k * T) + tks
        sk = s[k * ns:(k + 1) * ns]
        sm = []
        for i in range(M):
            si = np.sin(2 * np.pi * (f[i] * tk))
            sq = np.cos(2 * np.pi * (f[i] * tk))
            smi = np.sum(sk * si[:len(sk)])
            smq = np.sum(sk * sq[:len(sk)])
            sm.append(np.sqrt(smi ** 2 + smq ** 2))
        j = np.argmax(sm)
        xr = np.concatenate((xr, np.array(list(np.binary_repr(j, width=bps)), dtype=int)))
    # count errors
    x_reshaped = x.reshape(-1)
    xr_reshaped = xr.reshape(-1)
    err = np.not_equal(x_reshaped, xr_reshaped[:len(x_reshaped)])
    errors = np.sum(err)
    return errors


# BER simulation function
def simulate_ber(bps, Nsymb, ns, EbNo_values, coherent=True):
    ber = []
    for EbNo in EbNo_values:
        errors = 0
        for _ in range(1):  # averaging over iterations
            if coherent:
                errors += fsk_errors(bps, Nsymb, ns, EbNo)
            else:
                errors += fsk_errors_non_coh(bps, Nsymb, ns, EbNo)
        ber.append(errors / (Nsymb * bps))
    return ber


# Theoretical BER for coherent FSK systems
def theoretical_ber_coh(EbNo, M):
    if M == 2:
        return np.array([0.15866, 0.13093, 0.10403, 0.078896, 0.056495,
    0.037679, 0.023007, 0.012587, 0.0060044, 0.0024133,
    0.0007827, 0.00019399, 3.4303e-05, 3.9692e-06, 2.6951e-07])
    elif M == 4:
        return np.array([0.11814, 0.087789, 0.060786, 0.038512, 0.021824, 
    0.010751, 0.0044428, 0.0014733, 0.00037102, 6.6229e-05, 
    7.6892e-06, 5.2118e-07, 1.7997e-08, 2.6653e-10, 1.362e-12])
    elif M == 8:
        return np.array([0.10227, 0.067834, 0.041318, 0.022024, 0.0099156, 
    0.0036087, 0.0010058, 0.00020086, 2.6486e-05, 2.0836e-06, 
    8.6119e-08, 1.5924e-09, 1.074e-11, 2.0391e-14, 7.8524e-18])
    elif M == 16:
        return np.array([0.089859, 0.055663, 0.029957, 0.013469, 0.004819,
    0.001293, 0.00024205, 2.897e-05, 1.9921e-06, 6.8928e-08,
    1.0145e-09, 5.1257e-12, 6.7629e-15, 1.6453e-18, 4.6157e-23])
    elif M == 32:
        return np.array([0.082719, 0.047105, 0.022469, 0.0085348, 0.0024266,
    0.00047917, 6.0083e-05, 4.2975e-06, 1.5388e-07, 2.3425e-09,
    1.2294e-11, 1.6992e-14, 4.3826e-18, 1.3266e-22, 2.1688e-28])
    else:
        raise ValueError("Unsupported value of M for coherent case")

# Theoretical BER for non-coherent FSK systems
def theoretical_ber_non_coh(EbNo, M):
    if M == 2:
        return np.array([0.30327, 0.26644, 0.22637, 0.18438, 0.1424, 0.10287, 
    0.068311, 0.0408, 0.021324, 0.0094212, 0.003369, 
    0.0009231, 0.00018089, 2.3244e-05, 1.7558e-06])
    elif M == 4:
        return np.array([0.22934, 0.18475, 0.13987, 0.097719, 0.061557, 
    0.033946, 0.01579, 0.0059139, 0.0016837, 0.00033939, 
    4.4371e-05, 3.3753e-06, 1.3045e-07, 2.1593e-09, 1.233e-11])
    elif M == 8:
        return np.array([0.19472, 0.14559, 0.099187, 0.059806, 0.030757, 
    0.012878, 0.0041438, 0.00095467, 0.00014449, 1.2945e-05, 
    6.0428e-07, 1.2541e-08, 9.4638e-11, 2.0092e-13, 8.6607e-17])
    elif M == 16:
        return np.array([0.17469, 0.12169, 0.074737, 0.038861, 0.01625, 
    0.005127, 0.0011288, 0.00015786, 1.2538e-05, 4.943e-07, 
    8.2001e-09, 4.6432e-11, 6.8517e-14, 1.8682e-17, 6.0826e-22])
    elif M == 32:
        return np.array([0.16103, 0.10471, 0.058014, 0.025984, 0.0088058, 
    0.0020817, 0.00031127, 2.6219e-05, 1.0859e-06, 1.8789e-08, 
    1.1086e-10, 1.7154e-13, 4.9582e-17, 1.737e-21, 4.2722e-27])
    else:
        raise ValueError("Unsupported value of M for non-coherent case")


# Function to update the plot
def update_plot(bps):
    Nsymb = 2000  # number of symbols
    ns = 100  # oversampling factor
    EbNo_dB_sim = np.arange(0, 10, 2)  # Eb/No values in dB for simulation
    EbNo_dB_theory = np.arange(0, 15, 1)  # Eb/No values in dB for theoretical
    
    EbNo_sim = 10 ** (EbNo_dB_sim / 10)  # convert dB to linear for simulation
    EbNo_theory = 10 ** (EbNo_dB_theory / 10)  # convert dB to linear for theoretical
    M = 2 ** bps  # M-ary FSK

    # Simulate BER
    ber_coh_sim = simulate_ber(bps, Nsymb, ns, EbNo_dB_sim, coherent=True)
    ber_non_coh_sim = simulate_ber(bps, Nsymb, ns, EbNo_dB_sim, coherent=False)
    
    # Theoretical BER
    ber_coh_theory = theoretical_ber_coh(EbNo_theory, M)
    ber_non_coh_theory = theoretical_ber_non_coh(EbNo_theory, M)

    # Show plot
    plt.figure(figsize=(10, 6))
    plt.semilogy(EbNo_dB_sim, ber_coh_sim, 'o', label=f'Simulated Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_sim, ber_non_coh_sim, 's', label=f'Simulated Non-Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_theory, ber_coh_theory, label=f'Theoretical Coherent {M}-FSK')
    plt.semilogy(EbNo_dB_theory, ber_non_coh_theory, label=f'Theoretical Non-Coherent {M}-FSK')

    plt.title(f'BER vs. Eb/No for {M}-FSK Systems')
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.grid(True, which='both')
    plt.legend()
    plt.show()

# Dropdown for M-FSK selection
bps_dropdown = Dropdown(
    options=[('2-FSK', 1), ('4-FSK', 2), ('8-FSK', 3), ('16-FSK', 4), ('32-FSK', 5)],
    value=2,
    description='M-FSK:',
)

# Interactive plot
interactive_plot = interactive(update_plot, bps=bps_dropdown)

input_widgets = VBox([bps_dropdown], layout=Layout(width='auto'))
plot_output = interactive_plot.children[-1]  # The output plot

# Create a VBox that includes both the input widgets and the loading animation
inputs_and_loader = HBox([input_widgets], layout=Layout(align_items='center'))

# Create an HBox to hold everything in a horizontal layout
ui = VBox([inputs_and_loader, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)
